## Introduction

The Human Risky Decision-Making (HURD) Toolkit let's you easily fit risky choice models to data using a high-level API. This includes classic models like Expected Utility Theory (EU) and Prospect Theory (PT) along with contemporary ones, such as the Mixture Of Theories (MOT).

## Getting Started

Before we start fitting models, we'll need to load some human behavioral data.

We'll use the large dataset from Peterson et al. (2021), loaded into a `Dataset` object, here called `data`.

In [1]:
# import the dataloader
from hurd.internal_datasets import load_c13k_data

# import some data
data = load_c13k_data()
print(data)

Let's take a quick look at the data before we make use of it.

In [2]:
# how many choice problems in the dataset?
n_choice_problems = len(data)

# look at the first choice problem in the dataset
_, choice_problem_1, bRate1 = data.iloc(0)

print(
    "There are {} choice problems in this dataset, each of type {}.".format(
        n_choice_problems, type(choice_problem_1)
    ),
    end="\n\n",
)

print(
    "For example, the first choice problem is\n\n{}.".format(choice_problem_1),
    end="\n\n",
)

print(
    "Participants chose between gamble A and B. They chose B with a rate of {:.2f}.".format(
        bRate1
    )
)

There are 9831 choice problems in this dataset, each of type <class 'hurd.dataset.Problem'>.

For example, the first choice problem is

Problem i0-p1:
A: Outcomes: [26. -1.], Probabilities: [0.95 0.05], # of outcomes: 2
B: Outcomes: [21. 23.], Probabilities: [0.95 0.05], # of outcomes: 2.

Participants chose between gamble A and B. They chose B with a rate of 0.63.


## Fitting Models

Let's fit a model to the above data. First, we'll need an optimizer to do the parameter fitting.

We'll use a gradient-based optimizer called Adam (Kingma & Ba, 2014).

We specify at minimum learning rate `lr` and the number of parameter updates `n_iters` to perform.

In [3]:
from hurd.optimizers import Adam
optimizer = Adam(lr=0.01, n_iters=200)

Now we can import and initialize a model. Let's start with Prospect Theory.

We pass in the optimizer and specify a loss function (mean squared error).

In [4]:
# import the model
from hurd.models.psychophysical import ProspectTheoryModel

# initialize the model
pt = ProspectTheoryModel(
    optimizer=optimizer,
    loss_function="mse",
)

Using device cuda for model DecisionModelBase


By default, `ProspectTheoryModel` uses common utility and probability weighting functions from the classic model, but we could also pass them in as arguments, and there's plenty to pick from.

In [5]:
# same exact model, just more explicit
pt = ProspectTheoryModel(
    util_func="GeneralPowerLossAverseUtil",
    pwf="KT_PWF",
    optimizer=optimizer,
    loss_function="mse",
)

Using device cuda for model DecisionModelBase


Before we fit the model, let's divide the data into non-overlapping train (90%) and validation (10%) sets.

The `split` method returns a generator object that we can use to iterate through splits in a loop.

In [6]:
# data can be split with a method yeilding an iterator
splitter = data.split(p=0.9, n_splits=1, shuffle=True, random_state=1)
print(splitter)

<generator object Dataset.split at 0x7fefacd2a8c0>


In this case, we just want one split of the data. The result is two dataset objects.

In [7]:
(train_data, val_data) = list(splitter)[0]
print(train_data, val_data)

<hurd.dataset.Dataset object at 0x7ff06c08b290> <hurd.dataset.Dataset object at 0x7ff06c4dd910>


Now we are finally ready to fit the Prospect Theory model.

Loss for both the training and validation splits is reported for each epoch.

In [8]:
pt.fit(dataset=train_data, val_dataset=val_data)

[Epoch 1/200] Train Loss: 0.15614, Val Loss: 0.15019, Elapsed: 1.36s * New Best * 
[Epoch 2/200] Train Loss: 0.15184, Val Loss: 0.14714, Elapsed: 1.15s * New Best * 
[Epoch 3/200] Train Loss: 0.14759, Val Loss: 0.14419, Elapsed: 1.14s * New Best * 
[Epoch 4/200] Train Loss: 0.14335, Val Loss: 0.14086, Elapsed: 1.14s * New Best * 
[Epoch 5/200] Train Loss: 0.13903, Val Loss: 0.13720, Elapsed: 1.09s * New Best * 
[Epoch 6/200] Train Loss: 0.13464, Val Loss: 0.13311, Elapsed: 1.16s * New Best * 
[Epoch 7/200] Train Loss: 0.13015, Val Loss: 0.12868, Elapsed: 1.17s * New Best * 
[Epoch 8/200] Train Loss: 0.12556, Val Loss: 0.12405, Elapsed: 1.13s * New Best * 
[Epoch 9/200] Train Loss: 0.12089, Val Loss: 0.11929, Elapsed: 1.11s * New Best * 
[Epoch 10/200] Train Loss: 0.11615, Val Loss: 0.11438, Elapsed: 1.10s * New Best * 
[Epoch 11/200] Train Loss: 0.11135, Val Loss: 0.10930, Elapsed: 1.10s * New Best * 
[Epoch 12/200] Train Loss: 0.10649, Val Loss: 0.10409, Elapsed: 1.15s * New Best * 
[

Now let's train a more complex model, a Mixture Of Theories (MOT) from Peterson et al. (2021).

Note that it performs much better than Prospect Theory on this dataset.

In [9]:
from hurd.models.mixture_of_theories import MixtureOfTheories

mot = MixtureOfTheories(
    optimizer=optimizer,
    loss_function="mse",
)

mot.fit(dataset=train_data, val_dataset=val_data)

Using device cuda for model DecisionModelBase
Using device cuda for model DecisionModelBase
Using device cuda for model DecisionModelBase
Using device cuda for model DecisionModelBase
Using device cuda for model DecisionModelBase


[Epoch 1/200] Train Loss: 0.11413, Val Loss: 0.11290, Elapsed: 1.55s
[Epoch 2/200] Train Loss: 0.10674, Val Loss: 0.10638, Elapsed: 1.40s
[Epoch 3/200] Train Loss: 0.10145, Val Loss: 0.10229, Elapsed: 1.41s
[Epoch 4/200] Train Loss: 0.09710, Val Loss: 0.09874, Elapsed: 1.36s
[Epoch 5/200] Train Loss: 0.09286, Val Loss: 0.09500, Elapsed: 1.33s
[Epoch 6/200] Train Loss: 0.08855, Val Loss: 0.09109, Elapsed: 1.34s
[Epoch 7/200] Train Loss: 0.08427, Val Loss: 0.08707, Elapsed: 1.37s
[Epoch 8/200] Train Loss: 0.08006, Val Loss: 0.08297, Elapsed: 1.34s
[Epoch 9/200] Train Loss: 0.07592, Val Loss: 0.07883, Elapsed: 1.36s
[Epoch 10/200] Train Loss: 0.07188, Val Loss: 0.07466, Elapsed: 1.45s
[Epoch 11/200] Train Loss: 0.06793, Val Loss: 0.07050, Elapsed: 1.40s
[Epoch 12/200] Train Loss: 0.06410, Val Loss: 0.06636, Elapsed: 1.35s
[Epoch 13/200] Train Loss: 0.06038, Val Loss: 0.06228, Elapsed: 1.35s
[Epoch 14/200] Train Loss: 0.05678, Val Loss: 0.05831, Elapsed: 1.35s
[Epoch 15/200] Train Loss: 0.